In [ ]:
import psycopg2
import fdb
from dotenv import load_dotenv
import os

credentials = load_dotenv()

# Conexões
cur_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=os.getenv("FDB_PORT"),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
).cursor()

cur_orig = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASS"),
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    schema=os.getenv("PG_SCHEMA")
).cursor()

def commit():
    cur_dest.commit()

# Ferramentas

In [ ]:
global cadest
cadest = {}
cadest = {}  # Inicializar como dicionário vazio

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

# Compras

## Base

In [ ]:
# Cadunimedida
limpa_tabela(("cadunimedida"))

insert = cur_dest.prepare("insert into cadunimedida(sigla,descricao) values(?,?)")

cur_orig.execute("select distinct replace(unidade, ' ', 'NULL'), nome_unidemb from cgamercadorias order by 1")

for row in cur_orig:
    cur_dest.execute(insert, (row[0], row[1]))
commit()

# Grupo e Subgrupo
limpa_tabela(("cadgrupo", "cadsubgrupo"))

cur_orig.execute("""
select
	distinct
	to_char(grupo::integer, '000') grupo,
	subgrupo,
	a.descricao
from
	cgasubgrupos a
union all
select distinct
	to_char(grupo::integer, '000') grupo,
	concat(substring(mercadoria,1,1), substring(subgrupo,2,2)) subgrupo,
	b.descricao
	--, a.mercadoria
from
	cgc_vw_mercadorias a
left join cgasubgrupos b using (grupo, subgrupo)
where a.mercadoria not like '0%'                  
union all
	select
	distinct
	to_char(grupo::integer, '000') grupo,
	'000',
	a.descricao
from
	cgagrupos a
order by
	2,
	1
""")

for row in cur_orig:
    if row.subgrupo == '000':
        cur_dest.execute("insert into cadgrupo(grupo, nome) values(?, ?)", (row.grupo, row.descricao))
        commit()
    else:
        cur_dest.execute("insert into cadsubgrupo(grupo, subgrupo, nome) values(?, ?, ?)", (row.grupo, row.subgrupo, row.descricao))
commit()

# Cadest
limpa_tabela(("cadest"))
global cadest

insert = cur_dest.prepare("insert into cadest(grupo, subgrupo, codigo, cadpro, codreduz, disc1, ocultar, unid1, tipopro, usopro) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
select
	to_char(grupo::integer, '000') grupo,
	concat(substring(mercadoria,1,1), substring(subgrupo,2,2)) subgrupo,
	substring(mercadoria, 2, 3) codigo,
	concat(
		to_char(grupo::integer, '000'),'.',concat(substring(mercadoria,1,1), substring(subgrupo,2,2)),'.',substring(mercadoria, 2, 3)
	) cadpro,
	idespecificacao codreduz,
	descricao disc1,
	case when inativo = 0 then 'N' else 'S' end ocultar,
	unidade unid1
from
	cgc_vw_mercadorias
order by 1, 2, 3
""")

for row in cur_orig:
    cur_dest.execute(insert, (row.grupo, row.subgrupo, row.codigo, row.cadpro, row.codreduz, row.disc1, row.ocultar, row.unid1, 'P', 'C'))
    cadest[row.codreduz] = row.cadpro
commit()

